In [269]:
#imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [270]:
#instanciando spark 
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Estatistica_descritiva') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [271]:
#diretorios de trabalho
title_basics = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/IMDB - Internet_Movie_Database/raw_data/title_basics.tsv'
title_ratings = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/IMDB - Internet_Movie_Database/raw_data/title_ratings.tsv'
stage_path = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/IMDB - Internet_Movie_Database/stage_data/'

In [272]:
#leitura das duas bases de dados 'title_basics' e 'title_ratings'

title_basics_df = (
    spark
    .read
    .option('sep','\t')
    .csv(title_basics,header=True,inferSchema=True)
    .select(
        col('tconst').alias('key_id'),
        col('titleType').alias('title_type'),
        col('primaryTitle').alias('primary_title'),
        col('originalTitle').alias('original_title'),
        col('isAdult').alias('is_adult'),
        col('startYear').alias('start_year'),
        col('endYear').alias('end_year'),
        col('runtimeMinutes').alias('runtime_minutes'),
        col('genres').alias('genres')
    )
)

title_ratings_df = (
    spark
    .read
    .option('sep','\t')
    .csv(title_ratings,header=True,inferSchema=True)
    .select(
            col('tconst').alias('rating_id'),
            col('averageRating').alias('average_rating'),
            col('numVotes').alias('num_votes')
    )
)

print('dataframes readed!')


dataframes readed!


In [273]:
#Junção dos dataframes em um unico (JOIN)
joined_df = (
    title_basics_df
    .join(title_ratings_df,title_basics_df.key_id == title_ratings_df.rating_id,'full')
    .select(
        'key_id',
        'title_type',
        'primary_title',
        'original_title',
        'is_adult',
        'start_year',
        'end_year',
        'runtime_minutes',
        'genres',
        'average_rating',
        'num_votes'

    )
)

joined_df.show(truncate=False)

+---------+----------+--------------------------------------+--------------------------------------+--------+----------+--------+---------------+-------------------+--------------+---------+
|key_id   |title_type|primary_title                         |original_title                        |is_adult|start_year|end_year|runtime_minutes|genres             |average_rating|num_votes|
+---------+----------+--------------------------------------+--------------------------------------+--------+----------+--------+---------------+-------------------+--------------+---------+
|tt0000008|short     |Edison Kinetoscopic Record of a Sneeze|Edison Kinetoscopic Record of a Sneeze|0       |1894      |\N      |1              |Documentary,Short  |5.5           |1965     |
|tt0000015|short     |Autour d'une cabine                   |Autour d'une cabine                   |0       |1894      |\N      |2              |Animation,Short    |6.2           |947      |
|tt0000019|short     |The Clown Barber       

In [276]:
#escrita do arquivo com as devidas tranformaçoes na camada stage em formato parquet 
(
    joined_df
    .write
    .mode('overwrite')
    .format('parquet')
    .save(stage_path)
)

In [277]:
#Leitura do arquivo em formato parquet a partir da camada stage

movie_df = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .load(stage_path)
)

movie_df.show(truncate=False)

+---------+----------+---------------------------------------------------------+---------------------------------------------------------+--------+----------+--------+---------------+-----------------+--------------+---------+
|key_id   |title_type|primary_title                                            |original_title                                           |is_adult|start_year|end_year|runtime_minutes|genres           |average_rating|num_votes|
+---------+----------+---------------------------------------------------------+---------------------------------------------------------+--------+----------+--------+---------------+-----------------+--------------+---------+
|tt0000005|short     |Blacksmith Scene                                         |Blacksmith Scene                                         |0       |1893      |\N      |1              |Comedy,Short     |6.2           |2383     |
|tt0000029|short     |Baby's Meal                                              |Repas de béb

Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?

In [279]:
movie_operations =(
    movie_df
    .filter('start_year == 2015')
    .filter('title_type == "tvMovie" or title_type == "movie"')
    .count()
    
)

print(f'foram lançados {movie_operations} filmes em 2015.')


foram lançados 19987 filmes em 2015.


Qual o gênero de títulos mais frequente?

In [280]:

(
    movie_df
    .groupBy('genres')
    .count()
    .sort(col('count').desc())
    .limit(5)
    .show(truncate=False)
)


+-----------+------+
|genres     |count |
+-----------+------+
|Drama      |880649|
|\N         |643012|
|Comedy     |568956|
|Talk-Show  |467788|
|Documentary|392359|
+-----------+------+



Qual o gênero com a melhor nota média de títulos?    (errei)

In [281]:
(
    movie_df
    .sort(col('average_rating').desc())
    .groupBy('genres')
    .count()
    .sort(col('count').desc())
    .show(truncate=False)
)

+-----------------+------+
|genres           |count |
+-----------------+------+
|Drama            |880649|
|\N               |643012|
|Comedy           |568956|
|Talk-Show        |467788|
|Documentary      |392359|
|Drama,Romance    |354735|
|News             |266279|
|Reality-TV       |251369|
|Adult            |221116|
|Short            |169026|
|Drama,Short      |161288|
|Family           |136195|
|News,Talk-Show   |135690|
|Comedy,Short     |132338|
|Documentary,Short|125081|
|Game-Show        |122741|
|Music,Short      |97140 |
|Romance          |90742 |
|Music            |88697 |
|Sport            |88461 |
+-----------------+------+
only showing top 20 rows



Qual o vídeo game do gênero aventura mais bem avaliado em 2020?

In [282]:
#filtrando todos os games de 2020
games20 = (
    movie_df
    .filter('start_year == 2020 and title_type == "videoGame"')
)

#criando tabela sql
games20.createOrReplaceTempView('games20')

In [283]:
adventureGames20 = (
    spark
    .sql(
        'select * from games20 where games20.genres like "%Adventure%"'
    )
)

adventureGames20.show(truncate=False)

+----------+----------+---------------------------------------------------+---------------------------------------------------+--------+----------+--------+---------------+---------------------------+--------------+---------+
|key_id    |title_type|primary_title                                      |original_title                                     |is_adult|start_year|end_year|runtime_minutes|genres                     |average_rating|num_votes|
+----------+----------+---------------------------------------------------+---------------------------------------------------+--------+----------+--------+---------------+---------------------------+--------------+---------+
|tt10437042|videoGame |Predator: Hunting Grounds                          |Predator: Hunting Grounds                          |0       |2020      |\N      |\N             |Action,Adventure,Fantasy   |6.4           |160      |
|tt11585486|videoGame |Genshin Impact                                     |Genshin Impact       

In [284]:
(
    adventureGames20
    .sort(col('average_rating').desc())
    .select('primary_title','genres','average_rating')
    .limit(5)
    .show(truncate=False)
)

+-----------------------------+------------------------+--------------+
|primary_title                |genres                  |average_rating|
+-----------------------------+------------------------+--------------+
|Half-Life: Alyx              |Action,Adventure,Horror |9.5           |
|Ghost of Tsushima            |Action,Adventure,Drama  |9.3           |
|Omori                        |Adventure,Drama,Fantasy |9.2           |
|Ori and the Will of the Wisps|Adventure,Fantasy       |9.1           |
|Final Fantasy VII Remake     |Action,Adventure,Fantasy|9.1           |
+-----------------------------+------------------------+--------------+



Quantos títulos de filmes diferentes existem? Use df_titles.select('primaryTitle').distinct().count().

In [287]:
movie_distinct = (
    movie_df
    .select('primary_title')
    .distinct()
    .count()
)

print(f'existem {movie_distinct} filmes diferentes ')


existem 3931670 filmes diferentes 


Qual a duração média dos filmes com conteúdo adulto? Use uma combinação de filter() e describe().

In [288]:
(
    movie_df
    .filter('is_adult == 1')
    .select('runtime_minutes')
    .describe()
    .show(truncate=False)
)

+-------+-----------------+
|summary|runtime_minutes  |
+-------+-----------------+
|count  |250127           |
|mean   |92.79938555059914|
|stddev |57.18982244754781|
|min    |1                |
|max    |\N               |
+-------+-----------------+



Quantos filmes têm o título atual (“primary”) diferente do título original? Use uma combinação de filter, e count().

In [289]:
diferent_title =(
    movie_df
    .filter('primary_title != original_title')
    .count()
)

print(diferent_title,'filmes tem o titulo atual diferente do original')

125056 filmes tem o titulo atual diferente do original


Qual o filme que tem o nome mais longo?

In [290]:
(
    movie_df
    .select('key_id','title_type',"primary_title", length("primary_title").alias("title_size"))
    .sort(col('title_size').desc())
    .show()
)

+----------+----------+--------------------+----------+
|    key_id|title_type|       primary_title|title_size|
+----------+----------+--------------------+----------+
|tt12985206|     video|Otoko wa chi _ ko...|       419|
|tt13007080|     video|Creampie Voluntee...|       409|
| tt7727908|     video|Yufukuna kurashi ...|       408|
|tt12870546|     video|An Ultra K-Cup Ti...|       405|
|tt12866892|     video|I'm A Dirty Old M...|       401|
| tt7395478|     video|'Oppai dakenara s...|       400|
|tt13558536|     video|While Horsing Aro...|       386|
|tt13053838|     video|The Dumb Cunt Is ...|       383|
|tt13931914| tvEpisode|EE.UU. Se replieg...|       368|
| tt7509356|     video|Kanojo ni furaret...|       366|
|tt13278010|     video|This Son Was Secr...|       351|
|tt10813978| tvEpisode|Y- the total ease...|       347|
|tt13009172|     video|I'm the Silent Ty...|       340|
|tt14128648|     video|'No, No, No! Your...|       339|
|tt13356578|     video|An Up Close and P...|    

Qual filme tem a maior quantidade de votos? Dica: Use describe().
Grupo de escolhas da pergunta



In [291]:
(
    movie_df
    .select('key_id','primary_title','num_votes')
    .sort(col('num_votes').desc())
    .limit(10)
    .show(truncate=False)
)

+---------+-------------------------------------------------+---------+
|key_id   |primary_title                                    |num_votes|
+---------+-------------------------------------------------+---------+
|tt0111161|The Shawshank Redemption                         |2449517  |
|tt0468569|The Dark Knight                                  |2405191  |
|tt1375666|Inception                                        |2157649  |
|tt0137523|Fight Club                                       |1930108  |
|tt0110912|Pulp Fiction                                     |1898801  |
|tt0109830|Forrest Gump                                     |1893438  |
|tt0944947|Game of Thrones                                  |1861842  |
|tt0133093|The Matrix                                       |1743487  |
|tt0120737|The Lord of the Rings: The Fellowship of the Ring|1723007  |
|tt0167260|The Lord of the Rings: The Return of the King    |1701824  |
+---------+-------------------------------------------------+---

Qual é a menor nota média de um filme? Use describe().

In [292]:
(
    movie_df
    .select('average_rating')
    .describe()
    .show()
)

+-------+------------------+
|summary|    average_rating|
+-------+------------------+
|  count|           1182639|
|   mean| 6.917028357766062|
| stddev|1.3974964575775983|
|    min|               1.0|
|    max|              10.0|
+-------+------------------+

